# Simple propagation tests

In [ ]:
import os
from astropy.io import fits
import asdf

import hcipy
from matplotlib.colors import TwoSlopeNorm
import matplotlib.pyplot as plt
import numpy as np

from pastis.config import CONFIG_PASTIS
from pastis.simulators.scda_telescopes import HexRingAPLC
from pastis import util

In [ ]:
zerocen = TwoSlopeNorm(vcenter=0)

In [ ]:
pxsize = 1024
diameter = 7.258823341
wvln = 500e-9
imlamD = 1.2 * 32
fpm_rad = 3.5
fpm_px = 150
sampling = 4

pupil_grid = hcipy.make_pupil_grid(dims=pxsize, diameter=diameter)
#pupil_grid = hcipy.make_uniform_grid(pxsize, diameter)
lam_over_d = wvln / diameter

aper_fname = '/Users/ilaginja/repos/PASTIS/data/SCDA/2-Hex/masks/TelAp_LUVex_02-Hex_gy_ovsamp04_N1024.fits'
pup_read = hcipy.read_fits(aper_fname)
aperture = hcipy.Field(pup_read.ravel(), pupil_grid)

apod_fname = '/Users/ilaginja/repos/PASTIS/data/SCDA/2-Hex/solutions/2-Hex_robust_0px_N1024.fits'
apod_read = hcipy.read_fits(apod_fname)
apodizer = hcipy.Field(apod_read.ravel(), pupil_grid)

# Create a focal plane mask
samp_foc = fpm_px / (fpm_rad * 2)
#focal_grid_fpm = hcipy.make_focal_grid_from_pupil_grid(pupil_grid=pupil_grid, q=samp_foc, num_airy=fpm_rad, wavelength=wvln)
focal_grid_fpm = hcipy.make_focal_grid(samp_foc, num_airy=fpm_rad)
fpm = 1 - hcipy.circular_aperture(2 * fpm_rad * lam_over_d)(focal_grid_fpm)

# Create a focal plane grid for the detector
#focal_det = hcipy.make_focal_grid_from_pupil_grid(pupil_grid=pupil_grid, q=sampling, num_airy=imlamD, wavelength=wvln)
focal_det = hcipy.make_focal_grid(q=sampling, num_airy=imlamD)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
hcipy.imshow_field(aperture)
plt.subplot(1, 3, 2)
hcipy.imshow_field(apodizer)
plt.subplot(1, 3, 3)
hcipy.imshow_field(fpm)

In [ ]:
prop_method = hcipy.FraunhoferPropagator(pupil_grid, focal_det)

In [ ]:
wf_active_pupil = hcipy.Wavefront(aperture, wavelength=wvln)
apod_prop = hcipy.Apodizer(apodizer)
wf_apod = apod_prop(wf_active_pupil)

# Propagate pupil with apod, or pupil only
#wf_before_fpm = prop_method(wf_apod)
wf_before_fpm = prop_method(wf_active_pupil)

In [ ]:
hcipy.write_fits(wf_before_fpm.real, 'real_before_fpm_PUP_pastis.fits')
hcipy.write_fits(wf_before_fpm.imag, 'imag_before_fpm_PUP_pastis.fits')

In [ ]:
before_fpm_real = hcipy.read_fits('real_before_fpm_PUP_pastis.fits')
before_fpm_imag = hcipy.read_fits('imag_before_fpm_PUP_pastis.fits')

In [ ]:
plt.figure(figsize=(14, 7))

zerocen = TwoSlopeNorm(vcenter=0, vmin=-7.5e-5, vmax=1e-5)
plt.subplot(1, 2, 1)
plt.imshow(wf_before_fpm.real.shaped - before_fpm_real, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("prop of aperture only real, hcipy0.5.2 - hcipy0.4.0")
plt.colorbar()

zerocen = TwoSlopeNorm(vcenter=0, vmin=-4e-6, vmax=4e-6)
plt.subplot(1, 2, 2)
plt.imshow(wf_before_fpm.imag.shaped - before_fpm_imag, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("prop of aperture only imag, hcipy0.5.2 - hcipy0.4.0")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))
zerocen = TwoSlopeNorm(vcenter=0, vmin=-1e-8, vmax=1e-8)

plt.subplot(1, 2, 1)
plt.imshow(wf_before_fpm.real.shaped, cmap='RdBu', origin='lower', norm=zerocen)
plt.title("before FPM real, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(wf_before_fpm.imag.shaped, cmap='RdBu', origin='lower')
plt.title("before FPM imag, hcipy0.4.0 - hcipy0.5.2")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 7))

zerocen = TwoSlopeNorm(vcenter=0, vmin=-7.5e-5, vmax=1e-5)
plt.subplot(1, 2, 1)
plt.imshow(wf_before_fpm.real.shaped - wf_before_fpm3.real.shaped, cmap='RdBu', origin='lower')#, norm=zerocen)
plt.title("prop of aperture only real, hcipy0.5.2 - hcipy0.4.0")
plt.colorbar()

zerocen = TwoSlopeNorm(vcenter=0, vmin=-4e-6, vmax=4e-6)
plt.subplot(1, 2, 2)
plt.imshow(wf_before_fpm.imag.shaped - wf_before_fpm3.imag.shaped, cmap='RdBu', origin='lower')#, norm=zerocen)
plt.title("prop of aperture only imag, hcipy0.5.2 - hcipy0.4.0")
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 14))
plt.imshow(wf_before_fpm3.imag.shaped, cmap='RdBu', origin='lower')